<a href="https://colab.research.google.com/github/Che365/SDGP-/blob/Backend/SDGP_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
# Defining Categories
Categories = ['Caterpillar', 'Diabrotica speciosa', 'Healthy']
img_size = 50

In [ ]:
# Going through images and preporocessing
datadir = '/content/drive/MyDrive/SDGP Dataset'
data = []
for category in Categories:
    path = os.path.join(datadir, category)
    class_num = Categories.index(category)
    for img in os.listdir(path):
        try:
            img_array = tf.keras.preprocessing.image.load_img(os.path.join(path, img), target_size=(img_size, img_size))
            img_array = tf.keras.preprocessing.image.img_to_array(img_array)
            data.append([img_array, class_num])
        except Exception as e:
            print(f"Error processing {os.path.join(path, img)}: {str(e)}")

In [ ]:
X = np.array([item[0] for item in data])
y = np.array([item[1] for item in data])

# Normalize the pixel values
X = X / 255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=49)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(Categories), activation='softmax')
])

In [ ]:
# Comfiguring  the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training  the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
161/161 [==============================] - 22s 124ms/step - loss: 0.9274 - accuracy: 0.5620 - val_loss: 0.8535 - val_accuracy: 0.6373
Epoch 2/20
161/161 [==============================] - 18s 109ms/step - loss: 0.8158 - accuracy: 0.6525 - val_loss: 0.7332 - val_accuracy: 0.7005
Epoch 3/20
161/161 [==============================] - 18s 112ms/step - loss: 0.7370 - accuracy: 0.6825 - val_loss: 0.7094 - val_accuracy: 0.7067
Epoch 4/20
161/161 [==============================] - 20s 122ms/step - loss: 0.6707 - accuracy: 0.7155 - val_loss: 0.6660 - val_accuracy: 0.7340
Epoch 5/20
161/161 [==============================] - 18s 114ms/step - loss: 0.5947 - accuracy: 0.7520 - val_loss: 0.8114 - val_accuracy: 0.6654
Epoch 6/20
161/161 [==============================] - 18s 112ms/step - loss: 0.5420 - accuracy: 0.7843 - val_loss: 0.6543 - val_accuracy: 0.7176
Epoch 7/20
161/161 [==============================] - 21s 129ms/step - loss: 0.4436 - accuracy: 0.8276 - val_loss: 0.6162 - val_ac

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

41/41 [==============================] - 1s 25ms/step - loss: 1.2260 - accuracy: 0.7504
Test accuracy: 0.7503899931907654


In [ ]:
model.save('trained_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
Categories = ['Caterpillar', 'Diabrotica speciosa', 'Healthy']

# Load the saved model
model = tf.keras.models.load_model('/content/trained_model.h5')

# Load and preprocess the new image
url = '/content/healthy (1).jpg'
img = load_img(url, target_size=(50, 50))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class
predicted_class_index = np.argmax(predictions)
predicted_class = Categories[predicted_class_index]
confidence = predictions[0][predicted_class_index]

print(f"The predicted image is: {predicted_class} with confidence: {confidence * 100}%")


1/1 [==============================] - 0s 77ms/step
The predicted class is: Healthy with confidence: 99.33176040649414%
